In [1]:
import sys
import os.path
import numpy as np
import pandas as pd
import wave
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.backends.backend_pdf import PdfPages
from collections import OrderedDict
from scipy import signal
from scipy.optimize import curve_fit 
from functools import wraps

sys.path.append('/Users/tetsu/repo/research-tools/pyresearch/modules')

get_ipython().run_line_magic('matplotlib', 'inline')
%matplotlib inline

fig = plt.figure(figsize=(8, 11))

<Figure size 576x792 with 0 Axes>

In [2]:
from plot_tools import PlotTools


Window Not Found


In [5]:
N = 1024            # サンプル数
fs = 48000
dt = 1/fs            # サンプリング周期 [s]
f1, f2, f3 = 10, 60, 300 # 周波数 [Hz]

t = np.arange(0, N*dt, dt) # 時間 [s]
x = 3*np.sin(2*np.pi*f1*t) + 0.3*np.sin(2*np.pi*f2*t) + 0.2*np.sin(2*np.pi*f3*t) # 信号

### Design a filter

```python
scipy.signal.firwin(numtaps, cutoff, width=None, 
                    window='hamming', pass_zero=True, 
                    scale=True, nyq=None, fs=None)
```

In [6]:
filter1 = signal.firwin(numtaps=128, cutoff=5000, width=None, 
                            window='hamming', pass_zero=True, 
                            scaleTrue, nyq=None, fs=fs)

In [10]:
print(filter1/np.max(filter1) * (2**16 - 1))
print(np.max(filter1))
# df = pd.DataFrame(filter1/np.max(filter1) * (2**16 - 1))
df = pd.DataFrame(filter1/np.max(filter1))

[-8.46905106e+01 -8.59525092e+00  7.57539152e+01  1.35731227e+02
  1.44374043e+02  9.10180584e+01 -1.16104588e+01 -1.27676999e+02
 -2.08209412e+02 -2.09475712e+02 -1.13919229e+02  5.54858828e+01
  2.36231063e+02  3.46317319e+02  3.18535748e+02  1.35790886e+02
 -1.49543376e+02 -4.25370684e+02 -5.60464452e+02 -4.62846136e+02
 -1.31413934e+02  3.25438189e+02  7.19654486e+02  8.56820858e+02
  6.26137069e+02  6.74126573e+01 -6.20652378e+02 -1.14501093e+03
 -1.23778667e+03 -7.84146268e+02  9.95686980e+01  1.08164112e+03
  1.73226409e+03  1.70416925e+03  9.03545166e+02 -4.28265840e+02
 -1.77280940e+03 -2.52666212e+03 -2.26052151e+03 -9.38326055e+02
  1.00768470e+03  2.80025940e+03  3.61233243e+03  2.92782444e+03
  8.18081904e+02 -1.99971592e+03 -4.37917308e+03 -5.18191835e+03
 -3.77857717e+03 -4.07733657e+02  3.78103972e+03  7.06547963e+03
  7.78673861e+03  5.06718637e+03 -6.66897468e+02 -7.59163664e+03
 -1.29153136e+04 -1.37359581e+04 -8.06000125e+03  4.36988191e+03
  2.17502691e+04  4.05513

In [14]:
df

0
0   -0.001292
1   -0.000131
2    0.001156
3    0.002071
4    0.002203
5    0.001389
6   -0.000177
7   -0.001948
8   -0.003177
9   -0.003196
10  -0.001738
11   0.000847
12   0.003605
13   0.005284
14   0.004861
15   0.002072
16  -0.002282
17  -0.006491
18  -0.008552
19  -0.007063
20  -0.002005
21   0.004966
22   0.010981
23   0.013074
24   0.009554
25   0.001029
26  -0.009471
27  -0.017472
28  -0.018887
29  -0.011965
..        ...
98  -0.011965
99  -0.018887
100 -0.017472
101 -0.009471
102  0.001029
103  0.009554
104  0.013074
105  0.010981
106  0.004966
107 -0.002005
108 -0.007063
109 -0.008552
110 -0.006491
111 -0.002282
112  0.002072
113  0.004861
114  0.005284
115  0.003605
116  0.000847
117 -0.001738
118 -0.003196
119 -0.003177
120 -0.001948
121 -0.000177
122  0.001389
123  0.002203
124  0.002071
125  0.001156
126 -0.000131
127 -0.001292

[128 rows x 1 columns]

In [15]:
df.to_csv("lp_filt_128.csv", header=None, index=None)

In [23]:
tap = 4
filter5 = np.zeros(tap)
filter5[tap-1] = 1

In [24]:
filter5

array([0., 0., 0., 1.])

In [25]:
x = [1,2,3,4,5,6,7,8,9,10]

In [26]:
np.convolve(filter5,x,"same")

array([0., 0., 1., 2., 3., 4., 5., 6., 7., 8.])

### Adapt a filter
`scipy.signal.lfilter(b, a, x, axis=-1, zi=None)`
$$
\displaystyle  H(z)=\frac{\sum_{k=0}^{M} b_k z^{-k} } {\sum_{k=0}^{N}  a_k z^{-k} } 
$$

In [32]:
impulse = np.zeros(4096)
impulse[0] = 1

In [33]:
# y1 = signal.lfilter(filter1, 1, x)

In [34]:
y1 = signal.lfilter(filter1, 1, impulse)

In [35]:
F1 = np.fft.fft(y1)
Amp1 = np.abs(F1/(N/2))
Amp1[0] = Amp1[0]/2

In [37]:
plot_3charts(N=N, y=y1, fs=fs, start_sec=0, window_func_name="hamming")

IndexError: index 1024 is out of bounds for axis 0 with size 1024

In [22]:
y1_res = research_tools.ResearchTools(y=y1, fs=fs, fft_N=N, window="hamming")

NameError: name 'research_tools' is not defined

In [ ]:
y1_res.plot_y_time()

In [ ]:
y1_res.plot_freq_analysis_log()

In [ ]:
y1_res.plot_freq_analysis()

In [ ]:
y1_res.plot_power_gain_spectrum()

In [ ]:
y1_res.plot_acf()

In [ ]:
y1_res.acf

In [ ]:
plot_3charts(N=N, y=x, fs=fs, start_sec=0, window_func_name="hamming")

In [ ]:
filter2 = signal.firwin(numtaps=51, cutoff=100, fs=fs, pass_zero=False)
y2 = signal.lfilter(filter2, 1, x)

F2 = np.fft.fft(y2)
Amp2 = np.abs(F2/(N/2))
Amp2[0] = Amp2[0]/2

In [ ]:
plot_3charts(N=N, y=y2, fs=fs, start_sec=0, window_func_name="hamming")

In [ ]:
filter3 = signal.firwin(numtaps=51, cutoff=[30, 100], fs=1/dt, pass_zero=False)
y3 = signal.lfilter(filter3, 1, x)

F3 = np.fft.fft(y3)
Amp3 = np.abs(F3/(N/2))
Amp3[0] = Amp3[0] / 2
plot_3charts(N=N, y=y3, fs=fs, start_sec=0, window_func_name="hamming")

# クラスバージョンのテスト

In [ ]:
t2 = np.arange(0, N*dt, dt) # 時間 [s]
x2 = 300*np.sin(2*np.pi*10*t)

In [ ]:
x2_res = research_tools.ResearchTools(y=x2, N=N, fs=fs, window="hamming")

In [ ]:
x2_res.plot_freq_analysis()

In [ ]:
x2_res.plot_acf()

In [ ]:
x2_rec = research_tools.ResearchTools(y=x2, N=N, fs=fs, window="rectangular")

In [ ]:
x2_rec.plot_freq_analysis()

In [ ]:
x2_rec.plot_acf()

In [ ]:
x2_rec.plot_spectrogram_acf()

In [ ]:
x_rand = 300 * np.sin(2*np.pi*10*t2) + 200 * np.sin(2*np.pi*20*t2) + 200 * np.sin(2*np.pi*8*t2) + 50 * (np.random.rand(len(t2)) - 0.5)

In [ ]:
x_rres = research_tools.ResearchTools(y=x_rand, fft_N=N, fs=fs, window="rectangular")

In [ ]:
x_rres.plot_acf()

In [ ]:
x_rres.plot_spectrogram_acf()

In [ ]:
import wave_process

In [ ]:
wav = wave_process.wave_process("../../sample_wav/fukushima.wav")

In [ ]:
wav_r = research_tools.ResearchTools(y=wav.data[:524288], fft_N=524288, fs=wav.fs, window="hamming")

In [ ]:
wav_r.plot_acf()

In [ ]:
wav_r.plot_spectrogram_acf()

In [ ]:
filter5 = signal.firwin(numtaps=51, cutoff=8000, width=None, 
                            window='hamming', pass_zero=True, 
                            scale=True, nyq=None, fs=wav.fs)
wav_lpf = signal.lfilter(filter5, 1, wav.data[:524288])

In [ ]:
wav_rlpf = research_tools.ResearchTools(y=wav_lpf, fft_N=524288, fs=wav.fs, window="hamming")

In [ ]:
wav_rlpf.plot_spectrogram_acf()

In [ ]:
wav.wave_write("../../sample_wav/fuku_lpf4khz.wav", wav_lpf)

In [ ]:
wav_noise = wave_process.wave_process("../../sample_wav/fukushima.wav")

In [ ]:
print(wav_noise.data)
t = np.arange(0, wav_noise.chunk_size*1/wav_noise.fs, 1/wav_noise.fs) # 時間 [s]
wav_noise.data = wav_noise.data +  10 * np.sin(2*np.pi*5000*t)
print(wav_noise.data)

In [ ]:
wav_nres = research_tools.ResearchTools(y=wav.data[:524288], fft_N=524288, fs=wav_noise.fs, window="hamming")

In [ ]:
wav_nres.plot_spectrogram_acf()

In [ ]:
wav_noise.wave_write(filename="../../sample_wav/wav_noise.wav", data_array=wav_noise.data)